# Preprocessor Notebook : Logements Sociaux, fichier RPLS annuel

Ce notebook traite le fichier Excel du RPLS annuel : données sur les logements sociaux.
Le but est de récupérer les datasets suivants, à partir du fichier XSLX téléchargé depuis le site du ministère du Développement Durable :
 - Données par régions
 - Données par départements
 - Données par EPCI
 - Données par communes

 ### Paramètres
 Ce Notebook prend des paramètres en entrée, définis sur la toute première cellule (ci-dessus).
 La cellule a le tag "parameters" ce qui permet de lui passer des valeurs via papermill.
 - filepath : le chemin vers le fichier Excel à traiter
 - model_name : le nom du modèle source

 ### Principe
 Ce notebook extrait 4 feuilles du fichier Excel d'entrée : region, departement, epci, communes. 
 Chaque feuille est chargée dans un dataFrame, convertie en JSON, puis chargée en Bronze.

## Initialisation

Les cellules suivantes servent à importer les modules nécessaires et à préparer les variables communes utilisées dans les traitements.

In [1]:
# Baseline imports
import pandas as pd
import os
import sys
import datetime
# from dotenv import dotenv_values
# import sqlalchemy

# Dirty trick to be able to import common odis modules, if the notebook is not executed from 13_odis
current_dir = os.getcwd()
parent_dir = os.path.dirname(os.getcwd())
while not current_dir.endswith("13_odis"):
    print("changing to parent dir")
    os.chdir(parent_dir)
    current_dir = parent_dir
    parent_dir = os.path.dirname(current_dir)

print(os.getcwd())
sys.path.append(current_dir)

changing to parent dir
/Users/alex/dev/13_odis


In [2]:
# additional imports
from common.config import load_config
from common.data_source_model import DataSourceModel
from common.utils.file_handler import FileHandler
from common.utils.interfaces.data_handler import OperationType

## Paramètres du Notebook
Paramètres pouvant être passés en input par papermill.

Seuls des types built-in semblent marcher (str, int etc), les classes spécifiques ou les objets mutables (datetime...) semblent faire planter papermill.

Doc officielle de papermill : parametrize [https://papermill.readthedocs.io/en/latest/usage-parameterize.html]

In [3]:
# Define parameters for papermill. 
filepath = 'data/imports/logement/logement.logements_sociaux_1.xlsx'
model_name = "logement.logements_sociaux"


# Variables et fonctions utiles

Quelques variables et fonctions utilitaires sont définies ici.
Les fonctions utilitaires seront ultérieurement factorisées vers des classes Python dédiées.

In [4]:
# Initialize common variables
dataframes = {}
artifacts = []

start_time = datetime.datetime.now(tz=datetime.timezone.utc)
config = load_config("datasources.yaml", response_model=DataSourceModel)
model = config.get_model( model_name = model_name )
# Instantiate File Handler for file loads and dumps
handler = FileHandler()

In [5]:
import math

# Utility function to cleanup JSON data exported from a dataframe, before dumping it to json
def clean_json(obj):
    """
    Cleans JSON data by removing invalid values (e.g., NaN, INF, empty strings).
    
    :param obj: JSON object
    :return: Cleaned JSON object
    """
    if isinstance(obj, dict):
        return {k: clean_json(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [clean_json(v) for v in obj]
    elif isinstance(obj, float):
        return None if math.isinf(obj) or math.isnan(obj) else obj
    elif isinstance(obj, str):
        return None if obj.upper() in ("INF", "NA", "NAN", "") else obj
    return obj

## Traitement des données
A partir de là, on charge le fichier Excel dans Pandas et on traite les feuilles à récupérer, une par une

In [6]:
# Load workbook to pandas
wb = pd.ExcelFile(
    filepath,
    engine = 'openpyxl'
)

In [ ]:
# Load excel sheet for Regions
sheet_name = "REGION"
df_region = pd.read_excel(wb, 
                    sheet_name = "REGION",
                    index_col = "REG",
                    header = 5
                    )
dataframes["REGION"] = df_region

# Dump into a JSON artifact
region_json = df_region.to_dict(orient = 'records')
print(region_json)
region_artifact = handler.artifact_dump( region_json, "REGION", model, format = "json" )
artifacts.append(region_artifact)

df_region.head()

[{'REG': 1, 'LIBREG': 'Guadeloupe', 'nb_loues': 35657, 'nb_vacants': 1508, 'nb_vides': 1350, 'nb_asso': 161, 'nb_occup_finan': 1383, 'nb_occup_temp': 0, 'nb_ls': 40059, 'parc_non_conv': 0, 'nb_lgt_tot': 40059, 'densite': 22.67, 'nb_ls_en_qpv': 13836, 'nb_ls_individuels': 8318, 'nb_ls_collectifs': 31741, 'nb_ls_1piece': 1644, 'nb_ls_2piece': 5375, 'nb_ls_3piece': 20050, 'nb_ls_4piece': 11458, 'nb_ls_5piece_plus': 1532, 'nb_ls_plai': 5471, 'nb_ls_plus_ap_77': 26135, 'nb_ls_plus_av_77': 4886, 'nb_ls_pls': 1339, 'nb_ls_pli': 2228, 'nb_ls2023': 37505, 'nb_ls2023.1': 37505, 'nb_ls2022': 37380, 'nb_ls2021': 37202, 'nb_ls2020': 37208, 'nb_ls2019': 37004, 'nb_ls2018': 36221, 'nb_ls2017': 35546, 'nb_ls2016': 34855, 'nb_ls2015': 33453, 'nb_ls2014': 32216, 'nb_ls2013': 31637, 'evol_2023': 6.81, 'evol_2022': 0.33, 'evol_2022.1': 0.33, 'evol_2021': 0.48, 'evol_2020': -0.02, 'evol_2019': 0.55, 'evol_2018': 2.16, 'evol_2017': 1.9, 'evol_2016': 1.98, 'evol_2015': 4.19, 'evol_2014': 3.84, 'evol_2013': 1

,REG,LIBREG,nb_loues,nb_vacants,nb_vides,nb_asso,nb_occup_finan,nb_occup_temp,nb_ls,parc_non_conv,...,ener_A_new,ener_B_new,ener_C_new,ener_D_new,ener_E_new,ener_F_new,ener_G_new,ener_NR_new,nb_dpe_realise,perc_dpe_realise
0,1,Guadeloupe,35657,1508,1350,161,1383,0,40059,0,...,0,0,0,0,0,0,0,0,0,0.0
1,2,Martinique,33491,1156,269,19,506,0,35441,0,...,0,0,0,0,0,0,0,0,0,0.0
2,3,Guyane,19585,1213,405,0,559,0,21762,0,...,0,0,0,0,0,0,0,0,0,0.0
3,4,La Réunion,80140,1082,1583,188,470,0,83463,0,...,0,0,0,0,0,0,0,0,0,0.0
4,6,Mayotte,2234,248,78,60,321,0,2941,0,...,0,0,0,0,0,0,0,0,0,0.0


In [ ]:
# Load excel sheet for Departments
df_department = pd.read_excel(wb, 
                    sheet_name = "DEPARTEMENT",
                    index_col = "DEP",
                    header = 5
                    )
dataframes["DEPARTEMENT"] = df_department

# Dump into a JSON artifact
department_json = df_department.to_json()
department_artifact = handler.artifact_dump( department_json, "DEPARTEMENT", model, format = "json" )
artifacts.append(department_artifact)

df_department.head()

2025-04-08 22:32:50,073 - main - INFO :: file_handler.py :: logement.logements_sociaux -> results saved to : 'data/imports/logement/logement.logements_sociaux_DEPARTEMENT.xlsx'


,Unnamed: 1,densite,nb_ls,tx_vac,tx_mob
DEP,,,,,
01,Ain,17.41,49608,2.5346,9.4306
02,Aisne,17.72,41217,3.5452,10.1188
03,Allier,12.14,19853,5.1155,10.7542
04,Alpes-de-Haute-Provence,9.82,7888,2.0628,10.8499
05,Hautes-Alpes,11.80,8050,3.7698,7.6982


In [ ]:
# Load excel sheet for EPCI
df_epci = pd.read_excel(wb, 
                    sheet_name = "EPCI",
                    index_col = "EPCI_DEP",
                    header = 5
                    )

dataframes["EPCI"] = df_epci

# Dump into a JSON artifact
epci_json = df_epci.to_json()
epci_artifact = handler.artifact_dump( epci_json, "EPCI", model, format = "json" )
artifacts.append(epci_artifact)

df_epci.head()

2025-04-08 22:32:52,455 - main - INFO :: file_handler.py :: logement.logements_sociaux -> results saved to : 'data/imports/logement/logement.logements_sociaux_EPCI.xlsx'


,LIBEPCI,densite,nb_ls,tx_vac,tx_mob
EPCI_DEP,,,,,
200029999 - (01),CC Rives de l'Ain - Pays du Cerdon,9.90,637,2.7553,11.8506
200040350 - (01),CC Bugey Sud,12.68,1936,3.5656,9.9287
200040590 - (01),CA Villefranche Beaujolais Saône,26.53,713,1.8545,8.4165
200042497 - (01),CC Dombes Saône Vallée,12.37,1976,1.5041,10.2096
200042935 - (01),CA Haut - Bugey Agglomération,29.97,8178,3.7475,9.5453


In [ ]:
# Load excel sheet for COMMUNES
df_communes = pd.read_excel(wb, 
                    sheet_name = "COMMUNES",
                    index_col = "DEPCOM_ARM",
                    header = 5
                    )

dataframes["COMMUNES"] = df_communes

# Dump into a JSON artifact
communes_json = df_communes.to_json()
communes_artifact = handler.artifact_dump( communes_json, "COMMUNES", model, format = "json" )
artifacts.append(communes_artifact)

df_communes.head()

2025-04-08 22:33:25,282 - main - INFO :: file_handler.py :: logement.logements_sociaux -> results saved to : 'data/imports/logement/logement.logements_sociaux_COMMUNES.xlsx'


,LIBCOM_DEP,densite,nb_ls,tx_vac,tx_mob
DEPCOM_ARM,,,,,
01001,L'Abergement-Clémenciat (01),9.38,32,3.1250,9.0909
01004,Ambérieu-en-Bugey (01),30.42,2109,4.7952,10.1169
01005,Ambérieux-en-Dombes (01),14.22,113,2.7273,14.8515
01007,Ambronay (01),10.75,129,2.4390,12.1951
01008,Ambutrix (01),5.12,17,5.8824,5.8824


## Sauvegarde des métadonnées
On sauvegarde les métadonnées du processus localement, pour garder l'historique et pouvoir reprendre après erreur si besoin

In [10]:
for artifact in artifacts:
    print(artifact.model_dump( mode = "json" ))

preprocess_metadata = handler.dump_metadata(
    model = model,
    operation = OperationType.PREPROCESS,
    start_time = start_time,
    complete = True,
    errors = 0,
    artifacts = artifacts,
    pages = []
)

{'name': 'REGION', 'storage_info': {'location': 'data/imports/logement', 'format': 'xlsx', 'file_name': 'logement.logements_sociaux_REGION.xlsx', 'encoding': 'utf-8'}, 'load_to_bronze': True, 'success': True}
{'name': 'DEPARTEMENT', 'storage_info': {'location': 'data/imports/logement', 'format': 'xlsx', 'file_name': 'logement.logements_sociaux_DEPARTEMENT.xlsx', 'encoding': 'utf-8'}, 'load_to_bronze': True, 'success': True}
{'name': 'EPCI', 'storage_info': {'location': 'data/imports/logement', 'format': 'xlsx', 'file_name': 'logement.logements_sociaux_EPCI.xlsx', 'encoding': 'utf-8'}, 'load_to_bronze': True, 'success': True}
{'name': 'COMMUNES', 'storage_info': {'location': 'data/imports/logement', 'format': 'xlsx', 'file_name': 'logement.logements_sociaux_COMMUNES.xlsx', 'encoding': 'utf-8'}, 'load_to_bronze': True, 'success': True}
2025-04-08 22:33:29,300 - main - INFO :: file_handler.py :: logement.logements_sociaux -> results saved to : 'data/imports/logement/logement.logements_soc

## Chargement en couche Bronze
On instancie un JsonLoader pour charger tous les artifacts en base

In [ ]:
from common.utils.factory.loader_factory import create_loader

# instanciate json loader. 'format' is important here as model.format = 'xlsx'
loader = create_loader(config, model, handler=FileHandler(), format= "json")
loader.load_artifacts(artifacts)

# # prepare db client
# vals = dotenv_values()

# conn_str = "postgresql://{}:{}@{}:{}/{}".format(
#     vals["PG_DB_USER"],
#     vals["PG_DB_PWD"],
#     vals["PG_DB_HOST"],
#     vals["PG_DB_PORT"],
#     vals["PG_DB_NAME"]
# )

# dbengine = sqlalchemy.create_engine(conn_str)

In [ ]:
# # insert all to bronze
# # make the final table name lowercase to avoid issues in Postgre
# for name, dataframe in dataframes.items():
#     dataframe.to_sql(
#         name = f"{model.table_name}_{name.lower()}",
#         con = dbengine,
#         schema = 'bronze',
#         index = True,
#         if_exists = 'replace'
#     )
